In [ ]:
# !pip install tensorflow

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

pd.set_option("display.max_columns",None)

In [16]:
import os
import pandas as pd


main_folder = "/content/drive/MyDrive/data"

all_patients_data = []

for patient_folder in os.listdir(main_folder):
    patient_path = os.path.join(main_folder, patient_folder)

    if os.path.isdir(patient_path):

        blood_file = os.path.join(patient_path, "blood_monitoring.csv")
        bp_file = os.path.join(patient_path, "bp_monitoring.csv")
        lab_file = os.path.join(patient_path, "lab_results.csv")


        blood_df = pd.read_csv(blood_file)
        bp_df = pd.read_csv(bp_file)
        lab_df = pd.read_csv(lab_file)


        merged_df = blood_df.merge(bp_df, on=["Patient_ID", "Date", "Time"], how="outer")


        lab_columns = set(lab_df.columns) - {"Patient_ID", "Date"}


        existing_columns = lab_columns.intersection(merged_df.columns)


        lab_df = lab_df.drop(columns=existing_columns, errors="ignore")


        merged_df = merged_df.merge(lab_df, on=["Patient_ID", "Date"], how="outer")


        merged_df = merged_df.sort_values(by=["Patient_ID", "Date"], ascending=False)


        merged_df[list(existing_columns)] = merged_df[list(existing_columns)].ffill()


        all_patients_data.append(merged_df)
final_df = pd.concat(all_patients_data, ignore_index=True)
final_df=final_df.drop("Lab Result",axis=1)

final_output_path = "merged_patient_data.csv"
final_df.to_csv(final_output_path, index=False)

print(f"Final merged dataset saved at: {final_output_path}")


Final merged dataset saved at: merged_patient_data.csv


In [17]:
df=pd.read_csv("/content/merged_patient_data.csv")

In [18]:
df.head()

,Date,Time,Blood Glucose Level (mg/dL),Blood Oxygen (SpO₂),Electrocardiogram (ECG/EKG),Hydration Levels,Patient_ID,Blood Pressure,Heart Rate (HR),Respiratory Rate (RR),Body Temperature,Hemoglobin,Glucose,Cholesterol,Heart Rate,Platelet Count,WBC Count,RBC Count,Creatinine,Urea,Sodium,Potassium,Calcium
0,28-02-2025,0.07.33,125.3,95.4,1.18,67.0,P12346,95/84,62,15,36.9,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
1,28-02-2025,0.17.33,93.9,99.3,1.15,62.9,P12346,95/83,89,13,36.7,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
2,28-02-2025,0.27.33,185.0,97.2,0.89,59.4,P12346,122/67,69,17,37.7,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
3,28-02-2025,0.37.33,108.1,98.1,0.93,70.2,P12346,140/62,76,13,37.9,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
4,28-02-2025,0.47.33,127.6,97.3,0.89,55.9,P12346,135/80,60,19,36.9,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11


In [19]:
df.columns

Index(['Date', 'Time', 'Blood Glucose Level (mg/dL)', 'Blood Oxygen (SpO₂)',
       'Electrocardiogram (ECG/EKG)', 'Hydration Levels', 'Patient_ID',
       'Blood Pressure', 'Heart Rate (HR)', 'Respiratory Rate (RR)',
       'Body Temperature', 'Hemoglobin', 'Glucose', 'Cholesterol',
       'Heart Rate', 'Platelet Count', 'WBC Count', 'RBC Count', 'Creatinine',
       'Urea', 'Sodium', 'Potassium', 'Calcium'],
      dtype='object')

In [20]:
# preprocessing data
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
df['Timestamp'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d-%m-%Y %H.%M.%S')
df = df.drop(columns=['Date', 'Time','Glucose','Heart Rate'])
df = df.sort_values(by='Timestamp').reset_index(drop=True)
df[['Systolic_BP', 'Diastolic_BP']] = df['Blood Pressure'].str.split('/', expand=True).astype(float)
df = df.drop(columns=['Blood Pressure'])
df = df.fillna(method='ffill')



scaler = MinMaxScaler()
def normalize_group(group):
    df_numeric = group.drop(columns=['Patient_ID', 'Timestamp'])
    group[df_numeric.columns] = scaler.fit_transform(df_numeric)
    return group

df = df.groupby('Patient_ID', group_keys=False).apply(normalize_group)


def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

predictions = {}
for patient_id, group in df.groupby('Patient_ID'):
    data_values = group.drop(columns=['Patient_ID', 'Timestamp']).values


    X, y = create_sequences(data_values, seq_length=5)


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    model = Sequential([
        LSTM(50, activation='relu', return_sequences=True, input_shape=(5, X.shape[2])),
        LSTM(50, activation='relu'),
        Dense(X.shape[2])
    ])


    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print(f"Patient {patient_id} - Evaluation Metrics:")
    print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}, R² Score: {r2:.4f}")
    model.save(f"patient_{patient_id}_lstm_model.keras")
    last_sequence = X[-1].reshape(1, 5, X.shape[2])
    next_row_prediction = model.predict(last_sequence)
    next_row_original = scaler.inverse_transform(next_row_prediction)


    predictions[patient_id] = np.round(next_row_original, 2)

for patient_id, pred in predictions.items():
    print(f"Predicted Next Row for Patient {patient_id}:")
    print(np.round(pred, 2))

Epoch 1/20


<ipython-input-20-06615bceee63>:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')
<ipython-input-20-06615bceee63>:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Patient_ID', group_keys=False).apply(normalize_group)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 0.2498 - val_loss: 0.1110
Epoch 2/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0752 - val_loss: 0.1018
Epoch 3/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0539 - val_loss: 0.0981
Epoch 4/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0492 - val_loss: 0.0984
Epoch 5/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0479 - val_loss: 0.0958
Epoch 6/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0459 - val_loss: 0.0968
Epoch 7/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0450 - val_loss: 0.0930
Epoch 8/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0450 - val_loss: 0.0964
Epoch 9/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0447 - val_loss: 0.0985
Epoch 10/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0448 - val_loss: 0.0951
Epoch 11/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0442 - val_loss: 0.0956
Epoch 12/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0443 - val_loss: 0.0

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step
Patient P12332 - Evaluation Metrics:
MAE: 0.2701, RMSE: 0.3101, R² Score: -4.2090
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - loss: 0.2358 - val_loss: 0.1687
Epoch 2/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0551 - val_loss: 0.1635
Epoch 3/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0447 - val_loss: 0.1604
Epoch 4/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0446 - val_loss: 0.1621
Epoch 5/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0422 - val_loss: 0.1639
Epoch 6/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0433 - val_loss: 0.1642
Epoch 7/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0431 - val_loss: 0.1607
Epoch 8/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0430 - val_loss: 0.1568
Epoch 9/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0430 - val_loss: 0.1612
Epoch 10/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0423 - val_loss: 0.1619
Epoch 11/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0420 - val_loss: 0.1600
Epoch 12/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0422 - val_loss: 0

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step
Patient P12345 - Evaluation Metrics:
MAE: 0.3317, RMSE: 0.4016, R² Score: -3.3742
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.2340 - val_loss: 0.1441
Epoch 2/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0773 - val_loss: 0.1320
Epoch 3/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0543 - val_loss: 0.1233
Epoch 4/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0463 - val_loss: 0.1234
Epoch 5/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0467 - val_loss: 0.1266
Epoch 6/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0457 - val_loss: 0.1284
Epoch 7/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0459 - val_loss: 0.1275
Epoch 8/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0440 - val_loss: 0.1313
Epoch 9/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0439 - val_loss: 0.1302
Epoch 10/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0441 - val_loss: 0.1236
Epoch 11/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0440 - val_loss: 0.1263
Epoch 12/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0438 - val_loss: 0.1257

In [24]:
from tensorflow.keras.models import load_model
import numpy as np

seq_length = 5  # Same as used during training
predictions = {}

for patient_id, group in df.groupby('Patient_ID'):
    # Load the trained model for the specific patient
    model_path = f"patient_{patient_id}_lstm_model.keras"

    try:
        model = load_model(model_path)  # Load the saved model
    except Exception as e:
        print(f"Model not found for Patient {patient_id}, skipping...")
        continue

    # Get the last `seq_length` rows for this patient
    latest_data = group.drop(columns=['Patient_ID', 'Timestamp']).values

    if len(latest_data) < seq_length:
        print(f"Not enough data for Patient {patient_id}, skipping...")
        continue  # Skip if not enough data for sequence

    last_sequence = latest_data[-seq_length:].reshape(1, seq_length, latest_data.shape[1])  # Reshape for LSTM input

    # Make the prediction
    predicted_values = model.predict(last_sequence)

    # Reverse normalization to get original values
    predicted_values_original = scaler.inverse_transform(predicted_values)

    # Store the prediction
    predictions[patient_id] = np.round(predicted_values_original, 2)

# Print predictions for all patients
for patient_id, pred in predictions.items():
    print(f"Predicted Next Row for Patient {patient_id}:")
    print(pred)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
Predicted Next Row for Patient P12332:
[[1.286700e+02 9.716000e+01 1.000000e+00 6.135000e+01 8.062000e+01
  1.556000e+01 3.699000e+01 1.495000e+01 2.078400e+02 2.818058e+05
  4.439050e+03 5.060000e+00 9.200000e-01 2.595000e+01 1.391500e+02
  4.170000e+00 9.880000e+00 1.108900e+02 7.210000e+01]]
Predicted Next Row for Patient P12345:
[[1.218900e+02 9.631000e+01 9.900000e-01 6.022000e+01 7.508000e+01
  1.475000e+01 3.665000e+01 1.539000e+01 1.632100e+02 2.337636e+05
  5.759440e+03 5.250000e+00 7.600000e-01 2.320000e+01 1.396700e+02
  3.800000e+00 9.940000e+00 1.109000e+02 7.296000e+01]]
Predicted Next Row for Patient P12346:
[[1.128200e+02 9.682000e+01 9.700000e-01 5.982000e+01 7.335000e+01
  1.512000e+01 3.672000e+01 1.565000e+01 1.964000e+02 2.599741e+05
  6.610240e+03 4.920000e+00 8.100000e-01 2.415000e+01 1.384800e+02
  4.390000e+00 1.004000e+01 1.107800e+02 7.365000e+

In [25]:
df1=pd.read_csv("/content/merged_patient_data.csv")

In [26]:
df1.head()

,Date,Time,Blood Glucose Level (mg/dL),Blood Oxygen (SpO₂),Electrocardiogram (ECG/EKG),Hydration Levels,Patient_ID,Blood Pressure,Heart Rate (HR),Respiratory Rate (RR),Body Temperature,Hemoglobin,Glucose,Cholesterol,Heart Rate,Platelet Count,WBC Count,RBC Count,Creatinine,Urea,Sodium,Potassium,Calcium
0,28-02-2025,0.07.33,125.3,95.4,1.18,67.0,P12346,95/84,62,15,36.9,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
1,28-02-2025,0.17.33,93.9,99.3,1.15,62.9,P12346,95/83,89,13,36.7,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
2,28-02-2025,0.27.33,185.0,97.2,0.89,59.4,P12346,122/67,69,17,37.7,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
3,28-02-2025,0.37.33,108.1,98.1,0.93,70.2,P12346,140/62,76,13,37.9,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
4,28-02-2025,0.47.33,127.6,97.3,0.89,55.9,P12346,135/80,60,19,36.9,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11


In [27]:
df1.columns

Index(['Date', 'Time', 'Blood Glucose Level (mg/dL)', 'Blood Oxygen (SpO₂)',
       'Electrocardiogram (ECG/EKG)', 'Hydration Levels', 'Patient_ID',
       'Blood Pressure', 'Heart Rate (HR)', 'Respiratory Rate (RR)',
       'Body Temperature', 'Hemoglobin', 'Glucose', 'Cholesterol',
       'Heart Rate', 'Platelet Count', 'WBC Count', 'RBC Count', 'Creatinine',
       'Urea', 'Sodium', 'Potassium', 'Calcium'],
      dtype='object')